### Jupyter Notebook "2-TF_IDF_Cosine_&_Word2vec_WMD":  
* 3-Imports.
     * 3.1-Import librerias necesarias.
     * 3.2-Import de DFs del Jupyter Notebook anterior.
* 4-División DFs para usarlos en los distintos entrenamientos.
* 5-Realizando Comparaciones y obteniendo Similitudes.
     * 5.1- TF-IDF & Cosine Similarity.
     * 5.2- Word Embedding (Word2vec) & WMD.
* 6-Export del DF para usarlo en el siguiente Jupyter Notebook.

### 3-Imports.

### 3.1-Import librerias necesarias.

In [3]:
import pandas as pd
import numpy as np

### 3.2-Import de DFs del Jupyter Notebook anterior.

In [4]:
df_candidatos_loaded = pd.read_csv('DF_625_CVs.csv')
df_candidatos_loaded.shape

(625, 4)

In [5]:
df_Jobs_loaded = pd.read_csv('DF_20593_Job_Desc.csv')
df_Jobs_loaded.shape

(20593, 4)

### 4-División DFs para usarlos en los distintos entrenamientos.

###  5-Realizando Comparaciones y obteniendo Similitudes.

###### Creamos un nuevo DF concatenando los 2 dataframe de manera que se comparen todos los CVs con todos los Jobs:

In [32]:
#Nos quedarán 10 x 10 = 100 filas.
df_Jobs_and_Candidates = pd.merge(df_candidatos_f.assign(A=1), df_Jobs.assign(A=1), on='A').drop('A', 1)
df_Jobs_and_Candidates

,Candidate_Name,Content_CV,clean_Content_CV,tokens_Content_CV,Job_Title,Job_Description,clean_Job_Description,tokens_Job_Description
0,DataScientist_Karla_Lewis,\nKARLA LEWIS\nData Scientist\nCONTACT WORK EX...,data scientist brooklyn ny data scientist grub...,"[data_scientist, brooklyn, ny, data_scientist,...",Data Scientist,"Master’s degree or above in a STEM field, incl...",master degree stem field including limited com...,"[master, degree, stem, field, including, limit..."
1,DataScientist_Karla_Lewis,\nKARLA LEWIS\nData Scientist\nCONTACT WORK EX...,data scientist brooklyn ny data scientist grub...,"[data_scientist, brooklyn, ny, data_scientist,...",Data Scientist 2,"\nReporting to the Director, Data & Analytics,...",reporting director data analytics senior data ...,"[reporting, director, data, analytics, senior,..."
2,DataScientist_Karla_Lewis,\nKARLA LEWIS\nData Scientist\nCONTACT WORK EX...,data scientist brooklyn ny data scientist grub...,"[data_scientist, brooklyn, ny, data_scientist,...",HCM Consultant,\nThe Oracle Cloud HCM Absence Consultant will...,oracle cloud hcm absence consultant responsibl...,"[oracle, cloud, hcm, absence, consultant, resp..."
3,DataScientist_Karla_Lewis,\nKARLA LEWIS\nData Scientist\nCONTACT WORK EX...,data scientist brooklyn ny data scientist grub...,"[data_scientist, brooklyn, ny, data_scientist,...",HCM Consultant 2,4+ years of experience in PeopleSoft or Oracle...,peoplesoft oracle eb implementation support hc...,"[peoplesoft, oracle, eb, implementation, suppo..."
4,DataScientist_Karla_Lewis,\nKARLA LEWIS\nData Scientist\nCONTACT WORK EX...,data scientist brooklyn ny data scientist grub...,"[data_scientist, brooklyn, ny, data_scientist,...",Machine Learning Engineer,Leveraging the latest machine and deep learnin...,leveraging latest machine deep learning techni...,"[leveraging, latest, machine, deep_learning, t..."
...,...,...,...,...,...,...,...,...
6245,386-CONSULTANT,MARKETING CONSULTANT Summar...,marketing consultant value creator marketing e...,"[marketing, consultant, value, creator, market...",Machine Learning Engineer 2,Collaborate with a multidisciplinary team to g...,collaborate multidisciplinary team gain insigh...,"[collaborate, multidisciplinary, team, gain, i..."
6246,386-CONSULTANT,MARKETING CONSULTANT Summar...,marketing consultant value creator marketing e...,"[marketing, consultant, value, creator, market...",Security Specialist,Work in a fast-paced environment that combine ...,fast paced environment combine technical secur...,"[fast, paced, environment, combine, technical,..."
6247,386-CONSULTANT,MARKETING CONSULTANT Summar...,marketing consultant value creator marketing e...,"[marketing, consultant, value, creator, market...",Security Specialist 2,\n Handling incoming requests for assistanc...,handling incoming request assistance business ...,"[handling, incoming, request, assistance, busi..."
6248,386-CONSULTANT,MARKETING CONSULTANT Summar...,marketing consultant value creator marketing e...,"[marketing, consultant, value, creator, market...",Web Developer Full Stack,\n\n Graduate Degree in Information Technol...,graduate degree information technology similar...,"[graduate, degree, information, technology, si..."


In [36]:
#Exportamos el csv:
df_Jobs_and_Candidates.to_csv('DF_Exportado_6250_rows.csv',index = False)

In [37]:
#Si lo queremos importar:
df_Jobs_and_Candidates = pd.read_csv('DF_Exportado_6250_rows.csv')
df_Jobs_and_Candidates.shape

(6250, 8)

In [46]:
array_unicode_CV = df_Jobs_and_Candidates['clean_Content_CV'].astype('U').values
array_unicode_Jobs= df_Jobs_and_Candidates['clean_Job_Description'].astype('U').values

### 5.1- TF-IDF & Cosine Similarity.

In [54]:
#https://www.py4u.net/discuss/188191
    
#Calculate cosine similarity using tf-idf vectors, between search queries and matched documents.
#Acá mi search query es la columna clean_Content_CV y el matched documents son los clean_Job_Description.
#Cuando hacemos un transform estamos pasandole la query para que se fije los TF-IDF que tiene ya entrenados
#del modelo (cuando hicimos fit ya lo entrenamos... y lo hicimos con los cvs y los job descriptions). 

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import paired_cosine_distances as pcd

# Initialize an instance of tf-idf Vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Generate the tf-idf vectors for the corpus
#tfidf_vectorizer.fit(df_Jobs_and_Candidates['clean_Content_CV'] + " " + df_Jobs_and_Candidates['clean_Job_Description'])
tfidf_vectorizer.fit(array_unicode_CV+array_unicode_Jobs)

A = tfidf_vectorizer.transform(array_unicode_CV)
B = tfidf_vectorizer.transform(array_unicode_Jobs)

# compute and the cosine similarity:
cosine = 1 - pcd(A, B)

df_Jobs_and_Candidates['tfidf_cosine'] = cosine

df_Jobs_and_Candidates

,Candidate_Name,Content_CV,clean_Content_CV,tokens_Content_CV,Job_Title,Job_Description,clean_Job_Description,tokens_Job_Description,tfidf_cosine
0,DataScientist_Karla_Lewis,\nKARLA LEWIS\nData Scientist\nCONTACT WORK EX...,data scientist brooklyn ny data scientist grub...,"['data_scientist', 'brooklyn', 'ny', 'data_sci...",Data Scientist,"Master’s degree or above in a STEM field, incl...",master degree stem field including limited com...,"['master', 'degree', 'stem', 'field', 'includi...",0.060513
1,DataScientist_Karla_Lewis,\nKARLA LEWIS\nData Scientist\nCONTACT WORK EX...,data scientist brooklyn ny data scientist grub...,"['data_scientist', 'brooklyn', 'ny', 'data_sci...",Data Scientist 2,"\nReporting to the Director, Data & Analytics,...",reporting director data analytics senior data ...,"['reporting', 'director', 'data', 'analytics',...",0.115418
2,DataScientist_Karla_Lewis,\nKARLA LEWIS\nData Scientist\nCONTACT WORK EX...,data scientist brooklyn ny data scientist grub...,"['data_scientist', 'brooklyn', 'ny', 'data_sci...",HCM Consultant,\nThe Oracle Cloud HCM Absence Consultant will...,oracle cloud hcm absence consultant responsibl...,"['oracle', 'cloud', 'hcm', 'absence', 'consult...",0.021795
3,DataScientist_Karla_Lewis,\nKARLA LEWIS\nData Scientist\nCONTACT WORK EX...,data scientist brooklyn ny data scientist grub...,"['data_scientist', 'brooklyn', 'ny', 'data_sci...",HCM Consultant 2,4+ years of experience in PeopleSoft or Oracle...,peoplesoft oracle eb implementation support hc...,"['peoplesoft', 'oracle', 'eb', 'implementation...",0.019523
4,DataScientist_Karla_Lewis,\nKARLA LEWIS\nData Scientist\nCONTACT WORK EX...,data scientist brooklyn ny data scientist grub...,"['data_scientist', 'brooklyn', 'ny', 'data_sci...",Machine Learning Engineer,Leveraging the latest machine and deep learnin...,leveraging latest machine deep learning techni...,"['leveraging', 'latest', 'machine', 'deep_lear...",0.051246
...,...,...,...,...,...,...,...,...,...
6245,386-CONSULTANT,MARKETING CONSULTANT Summar...,marketing consultant value creator marketing e...,"['marketing', 'consultant', 'value', 'creator'...",Machine Learning Engineer 2,Collaborate with a multidisciplinary team to g...,collaborate multidisciplinary team gain insigh...,"['collaborate', 'multidisciplinary', 'team', '...",0.041610
6246,386-CONSULTANT,MARKETING CONSULTANT Summar...,marketing consultant value creator marketing e...,"['marketing', 'consultant', 'value', 'creator'...",Security Specialist,Work in a fast-paced environment that combine ...,fast paced environment combine technical secur...,"['fast', 'paced', 'environment', 'combine', 't...",0.071639
6247,386-CONSULTANT,MARKETING CONSULTANT Summar...,marketing consultant value creator marketing e...,"['marketing', 'consultant', 'value', 'creator'...",Security Specialist 2,\n Handling incoming requests for assistanc...,handling incoming request assistance business ...,"['handling', 'incoming', 'request', 'assistanc...",0.041386
6248,386-CONSULTANT,MARKETING CONSULTANT Summar...,marketing consultant value creator marketing e...,"['marketing', 'consultant', 'value', 'creator'...",Web Developer Full Stack,\n\n Graduate Degree in Information Technol...,graduate degree information technology similar...,"['graduate', 'degree', 'information', 'technol...",0.029146


### 5.2- Word Embedding (Word2vec) & WMD.

#### Usamos un Word2vec entrenado con nuestro vocabulario.


#### Unimos las columnas 'tokens_Job_Description' de 'df_Jobs' y 'tokens_Content_CV' de 'df_candidatos_f' generando una lista de listas, la cual será nuestro vocabulario que introduciremos a Word EMbedding como entrenamiento:

In [219]:
list_vocab_Jobs = df_Jobs['tokens_Job_Description']
#list_vocab_Jobs.tolist()

In [220]:
list_vocab_Jobs

0        [master_degree, stem, field, including_limited...
1        [reporting, director, data, analytics, senior,...
2        [oracle, cloud, hcm, absence, consultant, resp...
3        [peoplesoft, oracle_eb, implementation, suppor...
4        [leveraging_latest, machine, deep_learning, te...
                               ...                        
20588    [company, description, searching, talented, cr...
20589    [location_san, francisco, caterm, full_time, p...
20590    [take_pride, knowing, thousand, life, positive...
20591    [company, description, offer, youas, world_lea...
20592    [c_c, programmingdevelopment, win, programming...
Name: tokens_Job_Description, Length: 20593, dtype: object

In [221]:
list_vocab_Candid = df_candidatos_f['tokens_Content_CV']
#list_vocab_Candid.tolist()

In [222]:
list_vocab_Candid

0      [data_scientist, brooklyn, ny, data_scientist,...
1      [nlp, data_scientist, brooklyn, ny, nlp, data_...
2      [hcm, technical, consultant, working, oracle, ...
3      [sap_hcm, consultant, com, qwikresume, marshvi...
4      [kasey, vista, detroit, senior, software_engin...
                             ...                        
620    [consultant, owner, florida, international, un...
621    [pavithra, shetty, customer, oriented, princip...
622    [business, consultant, business, sale, operati...
623    [principal_consultant, supply_chain, logistics...
624    [marketing, consultant, value, creator, market...
Name: tokens_Content_CV, Length: 625, dtype: object

In [223]:
all_vocab = list_vocab_Candid.append(list_vocab_Jobs) #Unimos las listas.

In [224]:
all_vocab

0        [data_scientist, brooklyn, ny, data_scientist,...
1        [nlp, data_scientist, brooklyn, ny, nlp, data_...
2        [hcm, technical, consultant, working, oracle, ...
3        [sap_hcm, consultant, com, qwikresume, marshvi...
4        [kasey, vista, detroit, senior, software_engin...
                               ...                        
20588    [company, description, searching, talented, cr...
20589    [location_san, francisco, caterm, full_time, p...
20590    [take_pride, knowing, thousand, life, positive...
20591    [company, description, offer, youas, world_lea...
20592    [c_c, programmingdevelopment, win, programming...
Length: 21218, dtype: object

In [225]:
lista_all_vocab = all_vocab.tolist()
lista_all_vocab[0]

['data_scientist',
 'brooklyn',
 'ny',
 'data_scientist',
 'grubhub',
 'current',
 'new_york',
 'ny',
 'implemented',
 'various',
 'time_series',
 'forecasting',
 'technique',
 'predict',
 'surge',
 'customer',
 'order',
 'lower',
 'average',
 'customer',
 'wait',
 'time',
 'minute',
 'github',
 'deployed',
 'recommendation',
 'engine',
 'production',
 'conditionally',
 'recommend',
 'menu',
 'item',
 'based',
 'past',
 'order',
 'history',
 'increase',
 'average',
 'order',
 'size',
 'designed',
 'model',
 'portland',
 'pilot',
 'program',
 'increase',
 'incentive',
 'driver',
 'peak',
 'ordering',
 'hour',
 'resulting_increase',
 'driver',
 'b',
 'availability',
 'peak',
 'ordering',
 'time',
 'statistic',
 'rutgers',
 'university',
 'data_scientist',
 'new',
 'brunswick_nj',
 'adobe',
 'new_york',
 'ny',
 'worked',
 'product',
 'marketing',
 'team',
 'identify',
 'customer',
 'interaction',
 'free',
 'trial',
 'maximize',
 'likelihood',
 'conversion',
 'resulting',
 'conversion_rate

In [226]:
len(lista_all_vocab)

21218

#### Modelo word2vec:

In [227]:
from gensim.models import Word2Vec

#La librería Gensim nos provee una simple API al algoritmo de Google 'word2vec' el cual es usado para crear word embeddings.
#Como vimos previamente, word2vec son modelos de redes neuronales superficiales de dos capas que están entrenadas para
#reconstruir contextos lingüísticos de palabras. 

#Podemos traer word embeddings pre-entrenados con noticias de Google por ejemplo, donde se entrenarán con 3 millones de palabras.
#Pero, nosotros lo que hacemos es utilizar nuestro vocabulario para entrenar y crear nuestros propios embeddings.  

#La entrada de la red neuronal es una palabra representada como un vector “one-hot”, es decir, un vector con tantas posiciones 
#como tamaño tenga el vocabulario. 
#Por ejemplo, si queremos representar la palabra “sol” de un vocabulario de 4 palabras (el sol está cayendo), usaremos un vector
#de de esta dimensión (5) con cero en todas las posiciones menos la correspondiente a la palabra “sol” que tendrá 
#un uno --> [0,1,0,0]
#Y la  salida de la red neuronal será otro vector “one-hot” llamado “Word embedding” de las mismas 4 posiciones que representará
#las probabilidades de cada una de las palabras sean vecinas de la palabra representada en la entrada. [0.12, 1, 0.4, 0.1]
#--> Es 1 en la 2da posición ya que es la misma posición de la palabra. 

#Word embedding es una de las representaciones de un documento de vocabulario más popular. Es capaz de caputar los
#contextos de una palabra en el documento, similitud semantica y sintáctica, relaciones con otras palabras, etc.

#Mediante la siguiente línea de código entrenamos al modelo Word2Vec mediante nuestras 1275 palabras contenidas en “all_Sentences”:
model=gensim.models.Word2Vec(sentences=lista_all_vocab,min_count=2,workers=4,window=4,sg=0) #lo entrenamos con las all_sentences anterior.
#Ver como dividir en train y test. En nuestro caso igualmente no es necesario ya que no vamos a predecir palabras en base a contectos, 
#simplemente vamos a obtener las palabras más similares... entonces usamos TODAS las sentencias como entrenamiento. 
#model = Word2Vec(sentences=lista_all_vocab, size=200, window=4, min_count=1, workers=4)
#Parametros funcion Word2Vec:
#min_count : Ignore all the words where frequency of each word is less than min_count, default value is 5. As we wanted to add all words in corpus, so value we provided is 1.
#size: la longitud del vector denso para representar cada palabra. #VER.
#window: es el tamaño del "filtro" que se usa para analizar el contexto entre una palbras y las otras.  #CREO, VER. 
#workers: las iteraciones de entrenamiento.  #ESO ENTENDI, VER. 
#sg: es si utiliza el algoritmo de entrenamiento CBOW o skip gram. Por defecto estáen 0 (CBOW).

model.save("model")
wrds=list(model.wv.vectors)   #El modelo nos da un vocabulario con palabras con las cuales podemos consultar. 
print(len(wrds))

#ME devuelve 15069 palabras de vocabulario.

60855


In [181]:
#Cargamos el modelo:

#from gensim import models
#w2vec_model = models.KeyedVectors.load_word2vec_format('model', binary=True)
#w2vec_model

#from gensim.models import Word2Vec
#model = Word2Vec.load_word2vec_format('model', binary=True)

#word2vec = KeyedVectors.load_word2vec_format('final.model', binary=False)
#from gensim import models
#w2vec_model = models.KeyedVectors.load_word2vec_format('model', binary=True)
#w2vec_model = KeyedVectors.load('model')

In [228]:
model.wv.get_vector('machine_learning')

array([-9.35329318e-01,  3.30134690e-01, -1.52708888e+00, -9.32685360e-02,
       -2.32141781e+00,  1.13510227e+00,  1.92877603e+00, -1.08878505e+00,
       -3.70160900e-02, -1.94043148e+00,  6.59955204e-01, -1.36610961e+00,
       -8.03965092e-01, -1.07488823e+00,  2.45476127e+00, -2.40702701e+00,
        1.28087670e-01,  3.92293036e-01, -8.57815683e-01, -1.33548349e-01,
        7.01223239e-02, -8.53365541e-01,  2.36701274e+00,  9.49352503e-01,
       -7.31995851e-02,  1.82397389e+00, -5.87599277e-01, -7.96092868e-01,
        5.39072193e-02,  6.38300657e-01,  6.50366664e-01,  2.47248158e-01,
        8.03261697e-02, -2.32437849e+00, -1.85387123e+00,  1.27883121e-01,
       -2.56175542e+00,  3.37904096e-02,  7.70332158e-01, -2.53822785e-02,
       -8.98168743e-01,  1.01843905e+00,  2.21773219e+00,  3.25790495e-01,
        5.62601626e-01, -1.13821220e+00, -1.69144189e+00,  1.89986244e-01,
        1.64483571e+00,  8.70197773e-01,  2.62504839e-03,  1.89018309e-01,
        7.53653884e-01, -

In [237]:
model.wv.wmdistance('deep_learning', 'machine_learning')

0.3036559729920758

In [234]:
#Gracias a nuestros words embeddings obtenidos por el entrenamiento en el modelo word2vec,
#ahora podemos realizar funciones como 'most_similar'....testeamos nuestro modelo: 
ModeloPrueba=model.wv.most_similar("machine_learning")     

#La función most_similar busca palabras que esten semánticamente
#cercas (que sean parecidas) a una palabra dada (En este caso 'machine_learning').

In [235]:
print(ModeloPrueba)

[('deep_learning', 0.8448491096496582), ('artificial_intelligence', 0.8281981348991394), ('nlp', 0.8264880776405334), ('natural_language', 0.7937481999397278), ('predictive_analytics', 0.7921558618545532), ('algorithm', 0.7918980717658997), ('predictive', 0.7864595055580139), ('predictive_modeling', 0.7853081226348877), ('newsql', 0.7812016010284424), ('data_mining', 0.7807395458221436)]


In [238]:
#Función WMD que usaremos:
def WMD(tokens_CV_Candidate,tokens_Job_Desc):
    return (model.wv.wmdistance(tokens_CV_Candidate, tokens_Job_Desc))  #'wmdistance' return the word mover distance between two documents. 

In [239]:
#Como prueba obtenemos el WMD comparando para la 1ra fila (entre las columnas tokens_Content_CV y tokens_Job_Description)
first_job_descr = df_Jobs_and_Candidates.loc[0,'tokens_Job_Description']
first_content_cv = df_Jobs_and_Candidates.loc[0,'tokens_Content_CV']

wmd_result = round(WMD(first_job_descr,first_content_cv),3)
print(wmd_result)
 
similarity_wdm = round((1/(1+wmd_result)),3)  
print(similarity_wdm)

0.113
0.898


#### Usamos el Word2vec descargado.

In [84]:
#Link del cual descargamos el archivo: https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
EMBEDDING_FILE = '/home/fedricio/Desktop/Embeddings_Utilizados/Word2vec/GoogleNews-vectors-negative300.bin.gz'
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)
#Vemos si algunos bigramas estan: #word2vec["python"]

In [85]:
#Función WMD que usaremos:
def WMD(tokens_CV_Candidate,tokens_Job_Desc):
    return (word2vec.wmdistance(tokens_CV_Candidate, tokens_Job_Desc))  #'wmdistance' return the word mover distance between two documents. 

In [193]:
#Como prueba obtenemos el WMD comparando para la 1ra fila (entre las columnas tokens_Content_CV y tokens_Job_Description)
first_job_descr = df_Jobs_and_Candidates.loc[0,'tokens_Job_Description']
first_content_cv = df_Jobs_and_Candidates.loc[0,'tokens_Content_CV']

wmd_result = round(WMD(first_job_descr,first_content_cv),3)
print(wmd_result)
 
similarity_wdm = round((1/(1+wmd_result)),3)  
print(similarity_wdm)

0.052
0.951


In [240]:
#Aplicamos WMD para TODO el DF... entre el contenido del CV y la descripcion del puesto; 
#y el resultado lo guardamos en las columna 'WMD_Job_Desc'.

#df_Jobs_and_Candidates['WMD_Job_Desc'] = df_Jobs_and_Candidates.apply(lambda row: round(WMD(row['tokens_Content_CV'],row['tokens_Job_Description']),3), axis=1)
#APlicando lo de  --> similarity = 1 / (1 + distance):
#(#similarity = 1 / (1 + distance) https://groups.google.com/g/gensim/c/-pRZnsOEaPQ)
df_Jobs_and_Candidates['WMD_Job_Desc'] = df_Jobs_and_Candidates.apply(lambda row: round((1/(1+(WMD(row['tokens_Content_CV'],row['tokens_Job_Description'])))),3), axis=1)

In [241]:
df_Jobs_and_Candidates

,Candidate_Name,Content_CV,clean_Content_CV,tokens_Content_CV,Job_Title,Job_Description,clean_Job_Description,tokens_Job_Description,tfidf_cosine,WMD_Job_Desc
0,DataScientist_Karla_Lewis,\nKARLA LEWIS\nData Scientist\nCONTACT WORK EX...,data scientist brooklyn ny data scientist grub...,"['data_scientist', 'brooklyn', 'ny', 'data_sci...",Data Scientist,"Master’s degree or above in a STEM field, incl...",master degree stem field including limited com...,"['master', 'degree', 'stem', 'field', 'includi...",0.060513,0.898
1,DataScientist_Karla_Lewis,\nKARLA LEWIS\nData Scientist\nCONTACT WORK EX...,data scientist brooklyn ny data scientist grub...,"['data_scientist', 'brooklyn', 'ny', 'data_sci...",Data Scientist 2,"\nReporting to the Director, Data & Analytics,...",reporting director data analytics senior data ...,"['reporting', 'director', 'data', 'analytics',...",0.115418,0.903
2,DataScientist_Karla_Lewis,\nKARLA LEWIS\nData Scientist\nCONTACT WORK EX...,data scientist brooklyn ny data scientist grub...,"['data_scientist', 'brooklyn', 'ny', 'data_sci...",HCM Consultant,\nThe Oracle Cloud HCM Absence Consultant will...,oracle cloud hcm absence consultant responsibl...,"['oracle', 'cloud', 'hcm', 'absence', 'consult...",0.021795,0.884
3,DataScientist_Karla_Lewis,\nKARLA LEWIS\nData Scientist\nCONTACT WORK EX...,data scientist brooklyn ny data scientist grub...,"['data_scientist', 'brooklyn', 'ny', 'data_sci...",HCM Consultant 2,4+ years of experience in PeopleSoft or Oracle...,peoplesoft oracle eb implementation support hc...,"['peoplesoft', 'oracle', 'eb', 'implementation...",0.019523,0.905
4,DataScientist_Karla_Lewis,\nKARLA LEWIS\nData Scientist\nCONTACT WORK EX...,data scientist brooklyn ny data scientist grub...,"['data_scientist', 'brooklyn', 'ny', 'data_sci...",Machine Learning Engineer,Leveraging the latest machine and deep learnin...,leveraging latest machine deep learning techni...,"['leveraging', 'latest', 'machine', 'deep_lear...",0.051246,0.896
...,...,...,...,...,...,...,...,...,...,...
6245,386-CONSULTANT,MARKETING CONSULTANT Summar...,marketing consultant value creator marketing e...,"['marketing', 'consultant', 'value', 'creator'...",Machine Learning Engineer 2,Collaborate with a multidisciplinary team to g...,collaborate multidisciplinary team gain insigh...,"['collaborate', 'multidisciplinary', 'team', '...",0.041610,0.927
6246,386-CONSULTANT,MARKETING CONSULTANT Summar...,marketing consultant value creator marketing e...,"['marketing', 'consultant', 'value', 'creator'...",Security Specialist,Work in a fast-paced environment that combine ...,fast paced environment combine technical secur...,"['fast', 'paced', 'environment', 'combine', 't...",0.071639,0.937
6247,386-CONSULTANT,MARKETING CONSULTANT Summar...,marketing consultant value creator marketing e...,"['marketing', 'consultant', 'value', 'creator'...",Security Specialist 2,\n Handling incoming requests for assistanc...,handling incoming request assistance business ...,"['handling', 'incoming', 'request', 'assistanc...",0.041386,0.910
6248,386-CONSULTANT,MARKETING CONSULTANT Summar...,marketing consultant value creator marketing e...,"['marketing', 'consultant', 'value', 'creator'...",Web Developer Full Stack,\n\n Graduate Degree in Information Technol...,graduate degree information technology similar...,"['graduate', 'degree', 'information', 'technol...",0.029146,0.910


### Vemos como ejemplo a Federico Calonge:

In [242]:
is_Federico_Calonge =  df_Jobs_and_Candidates['Candidate_Name']=='HCM_Federico_Calonge'
new_DF = df_Jobs_and_Candidates[is_Federico_Calonge]
print(new_DF.shape)
new_DF

(10, 10)


,Candidate_Name,Content_CV,clean_Content_CV,tokens_Content_CV,Job_Title,Job_Description,clean_Job_Description,tokens_Job_Description,tfidf_cosine,WMD_Job_Desc
20,HCM_Federico_Calonge,\n ...,hcm technical consultant working oracle tool p...,"['hcm', 'technical', 'consultant', 'working', ...",Data Scientist,"Master’s degree or above in a STEM field, incl...",master degree stem field including limited com...,"['master', 'degree', 'stem', 'field', 'includi...",0.069849,0.904
21,HCM_Federico_Calonge,\n ...,hcm technical consultant working oracle tool p...,"['hcm', 'technical', 'consultant', 'working', ...",Data Scientist 2,"\nReporting to the Director, Data & Analytics,...",reporting director data analytics senior data ...,"['reporting', 'director', 'data', 'analytics',...",0.083858,0.906
22,HCM_Federico_Calonge,\n ...,hcm technical consultant working oracle tool p...,"['hcm', 'technical', 'consultant', 'working', ...",HCM Consultant,\nThe Oracle Cloud HCM Absence Consultant will...,oracle cloud hcm absence consultant responsibl...,"['oracle', 'cloud', 'hcm', 'absence', 'consult...",0.314486,0.901
23,HCM_Federico_Calonge,\n ...,hcm technical consultant working oracle tool p...,"['hcm', 'technical', 'consultant', 'working', ...",HCM Consultant 2,4+ years of experience in PeopleSoft or Oracle...,peoplesoft oracle eb implementation support hc...,"['peoplesoft', 'oracle', 'eb', 'implementation...",0.367655,0.932
24,HCM_Federico_Calonge,\n ...,hcm technical consultant working oracle tool p...,"['hcm', 'technical', 'consultant', 'working', ...",Machine Learning Engineer,Leveraging the latest machine and deep learnin...,leveraging latest machine deep learning techni...,"['leveraging', 'latest', 'machine', 'deep_lear...",0.085925,0.904
25,HCM_Federico_Calonge,\n ...,hcm technical consultant working oracle tool p...,"['hcm', 'technical', 'consultant', 'working', ...",Machine Learning Engineer 2,Collaborate with a multidisciplinary team to g...,collaborate multidisciplinary team gain insigh...,"['collaborate', 'multidisciplinary', 'team', '...",0.061773,0.930
26,HCM_Federico_Calonge,\n ...,hcm technical consultant working oracle tool p...,"['hcm', 'technical', 'consultant', 'working', ...",Security Specialist,Work in a fast-paced environment that combine ...,fast paced environment combine technical secur...,"['fast', 'paced', 'environment', 'combine', 't...",0.040321,0.899
27,HCM_Federico_Calonge,\n ...,hcm technical consultant working oracle tool p...,"['hcm', 'technical', 'consultant', 'working', ...",Security Specialist 2,\n Handling incoming requests for assistanc...,handling incoming request assistance business ...,"['handling', 'incoming', 'request', 'assistanc...",0.032068,0.891
28,HCM_Federico_Calonge,\n ...,hcm technical consultant working oracle tool p...,"['hcm', 'technical', 'consultant', 'working', ...",Web Developer Full Stack,\n\n Graduate Degree in Information Technol...,graduate degree information technology similar...,"['graduate', 'degree', 'information', 'technol...",0.021652,0.881
29,HCM_Federico_Calonge,\n ...,hcm technical consultant working oracle tool p...,"['hcm', 'technical', 'consultant', 'working', ...",Web Developer Full Stack 2,\n· Enter existing website codebases and exten...,enter existing codebases extend functionality ...,"['enter', 'existing', 'codebases', 'extend', '...",0.056635,0.920


Observamos altos valores de similitud para los puestos 'HCM Consultant' y 'HCM Consultant 2' (algo esperado).

In [245]:
is_Bradly_Johnston =  df_Jobs_and_Candidates['Candidate_Name']=='MLEngineer_Bradly_Johnston'
new_DF = df_Jobs_and_Candidates[is_Bradly_Johnston]
print(new_DF.shape)
new_DF

(10, 10)


,Candidate_Name,Content_CV,clean_Content_CV,tokens_Content_CV,Job_Title,Job_Description,clean_Job_Description,tokens_Job_Description,tfidf_cosine,WMD_Job_Desc
40,MLEngineer_Bradly_Johnston,"\nBradly Johnston\n435 Kasey Vista, Detroit\n+...",kasey vista detroit senior software engineer m...,"['kasey', 'vista', 'detroit', 'senior', 'softw...",Data Scientist,"Master’s degree or above in a STEM field, incl...",master degree stem field including limited com...,"['master', 'degree', 'stem', 'field', 'includi...",0.196025,0.924
41,MLEngineer_Bradly_Johnston,"\nBradly Johnston\n435 Kasey Vista, Detroit\n+...",kasey vista detroit senior software engineer m...,"['kasey', 'vista', 'detroit', 'senior', 'softw...",Data Scientist 2,"\nReporting to the Director, Data & Analytics,...",reporting director data analytics senior data ...,"['reporting', 'director', 'data', 'analytics',...",0.346808,0.910
42,MLEngineer_Bradly_Johnston,"\nBradly Johnston\n435 Kasey Vista, Detroit\n+...",kasey vista detroit senior software engineer m...,"['kasey', 'vista', 'detroit', 'senior', 'softw...",HCM Consultant,\nThe Oracle Cloud HCM Absence Consultant will...,oracle cloud hcm absence consultant responsibl...,"['oracle', 'cloud', 'hcm', 'absence', 'consult...",0.059270,0.898
43,MLEngineer_Bradly_Johnston,"\nBradly Johnston\n435 Kasey Vista, Detroit\n+...",kasey vista detroit senior software engineer m...,"['kasey', 'vista', 'detroit', 'senior', 'softw...",HCM Consultant 2,4+ years of experience in PeopleSoft or Oracle...,peoplesoft oracle eb implementation support hc...,"['peoplesoft', 'oracle', 'eb', 'implementation...",0.058595,0.923
44,MLEngineer_Bradly_Johnston,"\nBradly Johnston\n435 Kasey Vista, Detroit\n+...",kasey vista detroit senior software engineer m...,"['kasey', 'vista', 'detroit', 'senior', 'softw...",Machine Learning Engineer,Leveraging the latest machine and deep learnin...,leveraging latest machine deep learning techni...,"['leveraging', 'latest', 'machine', 'deep_lear...",0.286638,0.924
45,MLEngineer_Bradly_Johnston,"\nBradly Johnston\n435 Kasey Vista, Detroit\n+...",kasey vista detroit senior software engineer m...,"['kasey', 'vista', 'detroit', 'senior', 'softw...",Machine Learning Engineer 2,Collaborate with a multidisciplinary team to g...,collaborate multidisciplinary team gain insigh...,"['collaborate', 'multidisciplinary', 'team', '...",0.333604,0.941
46,MLEngineer_Bradly_Johnston,"\nBradly Johnston\n435 Kasey Vista, Detroit\n+...",kasey vista detroit senior software engineer m...,"['kasey', 'vista', 'detroit', 'senior', 'softw...",Security Specialist,Work in a fast-paced environment that combine ...,fast paced environment combine technical secur...,"['fast', 'paced', 'environment', 'combine', 't...",0.064902,0.918
47,MLEngineer_Bradly_Johnston,"\nBradly Johnston\n435 Kasey Vista, Detroit\n+...",kasey vista detroit senior software engineer m...,"['kasey', 'vista', 'detroit', 'senior', 'softw...",Security Specialist 2,\n Handling incoming requests for assistanc...,handling incoming request assistance business ...,"['handling', 'incoming', 'request', 'assistanc...",0.044034,0.898
48,MLEngineer_Bradly_Johnston,"\nBradly Johnston\n435 Kasey Vista, Detroit\n+...",kasey vista detroit senior software engineer m...,"['kasey', 'vista', 'detroit', 'senior', 'softw...",Web Developer Full Stack,\n\n Graduate Degree in Information Technol...,graduate degree information technology similar...,"['graduate', 'degree', 'information', 'technol...",0.048537,0.905
49,MLEngineer_Bradly_Johnston,"\nBradly Johnston\n435 Kasey Vista, Detroit\n+...",kasey vista detroit senior software engineer m...,"['kasey', 'vista', 'detroit', 'senior', 'softw...",Web Developer Full Stack 2,\n· Enter existing website codebases and exten...,enter existing codebases extend functionality ...,"['enter', 'existing', 'codebases', 'extend', '...",0.066777,0.927


### 6-Export del DF para usarlo en el siguiente Jupyter Notebook.

In [246]:
#Exportamos el csv:
df_Jobs_and_Candidates.to_csv('DF_Exportado_Jobs_And_Candidates_own_model.csv',index = False)
#Si lo queremos importar:
DF_J_and_C = pd.read_csv('DF_Exportado_Jobs_And_Candidates_own_model.csv')
DF_J_and_C.tail()

,Candidate_Name,Content_CV,clean_Content_CV,tokens_Content_CV,Job_Title,Job_Description,clean_Job_Description,tokens_Job_Description,tfidf_cosine,WMD_Job_Desc
6245,386-CONSULTANT,MARKETING CONSULTANT Summar...,marketing consultant value creator marketing e...,"['marketing', 'consultant', 'value', 'creator'...",Machine Learning Engineer 2,Collaborate with a multidisciplinary team to g...,collaborate multidisciplinary team gain insigh...,"['collaborate', 'multidisciplinary', 'team', '...",0.041610,0.927
6246,386-CONSULTANT,MARKETING CONSULTANT Summar...,marketing consultant value creator marketing e...,"['marketing', 'consultant', 'value', 'creator'...",Security Specialist,Work in a fast-paced environment that combine ...,fast paced environment combine technical secur...,"['fast', 'paced', 'environment', 'combine', 't...",0.071639,0.937
6247,386-CONSULTANT,MARKETING CONSULTANT Summar...,marketing consultant value creator marketing e...,"['marketing', 'consultant', 'value', 'creator'...",Security Specialist 2,\n Handling incoming requests for assistanc...,handling incoming request assistance business ...,"['handling', 'incoming', 'request', 'assistanc...",0.041386,0.910
6248,386-CONSULTANT,MARKETING CONSULTANT Summar...,marketing consultant value creator marketing e...,"['marketing', 'consultant', 'value', 'creator'...",Web Developer Full Stack,\n\n Graduate Degree in Information Technol...,graduate degree information technology similar...,"['graduate', 'degree', 'information', 'technol...",0.029146,0.910
6249,386-CONSULTANT,MARKETING CONSULTANT Summar...,marketing consultant value creator marketing e...,"['marketing', 'consultant', 'value', 'creator'...",Web Developer Full Stack 2,\n· Enter existing website codebases and exten...,enter existing codebases extend functionality ...,"['enter', 'existing', 'codebases', 'extend', '...",0.038411,0.936
